Merging the e.coli and weather data 

Loading the data

In [20]:
import pandas as pd

# Load data
beach_data = pd.read_parquet("..\\data\\cleaned\\cleaned_hanlans_point_beach.parquet")
weather_data = pd.read_parquet("..\\data\\cleaned\\cleaned_toronto_city.parquet")

print(beach_data.shape)
print(weather_data.shape)



(1671, 6)
(78900, 10)


Align data while keeping enough for lagged variables

In [21]:
# Get relevant date range from beach_data
srt_date = beach_data['CollectionDate'].min() - pd.Timedelta(days=7) # Keep 7 days prior to start beach_data for lagged variables
end_date = beach_data['CollectionDate'].max()

print(f"start date = {srt_date}")
print(f"end date = {end_date}")

btwn_srt_end_dates = (weather_data['Date/Time'] >= srt_date) & (weather_data['Date/Time'] <= end_date)
weather_data_filtered = weather_data.loc[btwn_srt_end_dates]

print(weather_data_filtered.shape)

start date = 2008-08-16 00:00:00
end date = 2024-09-07 00:00:00
(70404, 10)


Merge data

In [19]:
merged = beach_data.merge(
    weather_data,
    how="inner",
    left_index=True,
    right_index=True
)

merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 703 entries, 120 to 1670
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   CollectionDate          703 non-null    datetime64[ns]
 1   beachId                 703 non-null    float64       
 2   beachName               703 non-null    object        
 3   eColi                   665 non-null    float64       
 4   advisory                703 non-null    object        
 5   statusFlag              703 non-null    object        
 6   Station Name            703 non-null    object        
 7   Date/Time               703 non-null    datetime64[ns]
 8   Max Temp (°C)           703 non-null    float64       
 9   Min Temp (°C)           703 non-null    float64       
 10  Mean Temp (°C)          703 non-null    float64       
 11  Heat Deg Days (°C)      703 non-null    float64       
 12  Cool Deg Days (°C)      703 non-null    float64     